In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification


In [2]:
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

In [7]:
from transformers import pipeline

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
nlp = nlp("Apple est créée le 1er avril 1976 dans le garage de la maison d'enfance de Steve Jobs à Los Altos en Californie par Steve Jobs retiré le 9 janvier 2015.")
nlp

[{'entity_group': 'ORG',
  'score': 0.99379635,
  'word': 'Apple',
  'start': 0,
  'end': 5},
 {'entity_group': 'PER',
  'score': 0.99533767,
  'word': 'Steve Jobs',
  'start': 74,
  'end': 85},
 {'entity_group': 'LOC',
  'score': 0.99801445,
  'word': 'Los Altos',
  'start': 87,
  'end': 97},
 {'entity_group': 'LOC',
  'score': 0.9980428,
  'word': 'Californie',
  'start': 100,
  'end': 111},
 {'entity_group': 'PER',
  'score': 0.9947664,
  'word': 'Steve Jobs',
  'start': 115,
  'end': 126}]

In [14]:
import pandas as pd
df = pd.DataFrame(nlp)
df = df.rename(columns={'entity_group': 'entity'})
count_df = df.groupby('entity').count().reset_index()[['entity', 'score']]
count_df = count_df.rename(columns={'score': 'count'})

  entity     score        word  start  end
0    ORG  0.993796       Apple      0    5
1    PER  0.995338  Steve Jobs     74   85
2    LOC  0.998014   Los Altos     87   97
3    LOC  0.998043  Californie    100  111
4    PER  0.994766  Steve Jobs    115  126


In [15]:
    from transformers import AutoTokenizer, AutoModelForTokenClassification
    tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
    model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")
    from transformers import pipeline

    nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")
    nlp_camem = (nlp(input_text))
    df = pd.DataFrame(nlp_camem)
    df = df.rename(columns={'entity_group': 'entity'})
    count_df = df.groupby('entity').count().reset_index()[['entity', 'score']]
    count_df = count_df.rename(columns={'score': 'count'})
    st.dataframe(count_df)

In [16]:
count_df

,entity,count
0,LOC,2
1,ORG,1
2,PER,2
